In [477]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import requests
import json

statesofInterest = ['IOWA', 'INDIANA', 'ILLINOIS', 'KANSAS', 'MINNESOTA', 'MISSOURI', 'NEBRASKA', 'NORTH DAKOTA', 'SOUTH DAKOTA', 'OHIO']

In [478]:
def crop_yield(year_param):

     # These are the parameters in the API that you choose.
    ploads = {'format': 'JSON',
              'soucre_desc': 'SURVEY',
              'sector_desc': 'CROPS',
              'group_desc': 'FIELD CROPS',
              'commodity_desc': 'SOYBEANS',
              'prodn_practice_desc': 'ALL',
              'statisticcat_desc': 'CONDITION',
              'state_name': ['IOWA', 'INDIANA', 'ILLINOIS', 'KANSAS', 'MINNESOTA', 'MISSOURI', 'NEBRASKA', 'NORTH DAKOTA', 'SOUTH DAKOTA', 'OHIO'],
              'year__GT': year_param,
              'prodn_practice_desc': 'ALL PRODUCTION PRACTICES'
              }
    # key (make a parameter)
    r = requests.get("https://quickstats.nass.usda.gov/api/api_GET/?key=1C757E50-5169-30CC-BEFD-40A5C3E2A43D", params=ploads)
    # execute get request
    dt1 = json.dumps(r.json(), indent=2, sort_keys=True)
    # make it json format
    obj1 = json.loads(dt1)

    # collect relevant data for each object created
    dt2 = []
    for i in obj1["data"]:
        dt2.append([i["year"], i["reference_period_desc"], i["commodity_desc"], i["Value"], i['unit_desc'], i['state_name']])
        #dt2.append([i])
    # convert from dictionary format to pandas
    dt3 = pd.DataFrame.from_dict(dt2)
    return dt3

In [479]:
ts = crop_yield('1800')
ts.columns = ["Year", "Week", "Crop", "Value", "Quality_Prct", "State"]
ts.sort_values(["Year", "Quality_Prct"], inplace=True)
ts.sort_values(["Year", "Week"], inplace=True)

In [480]:
state = ts["State"].unique()
qual  = ts["Quality_Prct"].unique()
year  = ts["Year"].unique()
weeks = ["WEEK #30", "WEEK #35", "WEEK #40"]

st1 = []
for s in state:
    stat1 = ts[ts["State"] == s]
    yr1 = []
    for y in year:
        year1 = stat1[stat1["Year"] == y]
        q1 = []
        for q in qual:
            qual1 = year1[year1["Quality_Prct"] == q] 
            new1 = qual1.pivot(index='Year', columns="Week", values='Value')
            
            avail1 = new1.columns
            test1 = weeks
            missing1 = np.setdiff1d(test1,avail1)
            
            for i in missing1:
                new1[i] = np.nan
            new1 = new1[weeks]
            
            new1.columns = [s + q for s in weeks] 
            q1.append(new1)
        qperyr = pd.concat(q1, axis=1) 
        yr1.append(qperyr)
    yrsq = pd.concat(yr1, axis = 0)
    yrsq["State"] = s
    st1.append(yrsq)
dt1 = pd.concat(st1, axis = 0)  
dt1.sort_values([ "State", "Year"], inplace=True) 


In [481]:
dt1.head()
bokeh = dt1[dt1["State"] == "IOWA"]
bokeh = bokeh["WEEK #35PCT GOOD"]
bokeh.to_csv("Iowa_NVDI.csv")

In [482]:
dt1['State'] = dt1['State'].str.upper() 
model_allStates_quality = dt1[dt1['State'].isin(statesofInterest)]
model_allStates_quality.to_csv("data_model_ready/model_allStates_quality.csv")